In [1]:
import pandas as pd
import re
import csv
from nltk import word_tokenize,pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk import stem
from nltk import sent_tokenize


import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline
doc = nlp("Barack Obama was born in Hawaii.") # run annotation over a sentence
from sklearn.model_selection import train_test_split

2024-08-14 09:46:46 INFO: Downloaded file to /Users/gbaldonado/stanza_resources/resources.json
2024-08-14 09:46:46 INFO: Downloading default packages for language: en (English) ...
2024-08-14 09:46:47 INFO: File exists: /Users/gbaldonado/stanza_resources/en/default.zip
2024-08-14 09:46:50 INFO: Finished downloading models and saved to /Users/gbaldonado/stanza_resources
2024-08-14 09:46:50 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-08-14 09:46:50 INFO: Downloaded file to /Users/gbaldonado/stanza_resources/resources.json
2024-08-14 09:46:52 INFO: Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

2024-08-14 09:46:52 INFO: Using device: cpu
2024-08-14 09:46:52 INFO: Loading: tokenize
2024-08-14 09:46:52 INFO: Loading: mwt
2024-08-14 09:46:52 INFO: Loading: pos
2024-08-14 09:46:52 INFO: Loading: lemma
2024-08-14 09:46:52 INFO: Loading: constituency
2024-08-14 09:46:53 INFO: Loading: depparse
2024-08-14 09:46:53 INFO: Loading: sentiment
2024-08-14 09:46:53 INFO: Loading: 

##### Stanford Core NLP
* We need to download the [Stanford core nlp](https://stanfordnlp.github.io/CoreNLP/download.html) package before we can start with the dataset creation process.  
``
cd /Users/krishns18/SFSU/Modeling_ALMA/corenlp/stanford-corenlp-4.0.0
java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer
``   
* The server will start and the command prompt will display like below   
``
(base) Shaileshs-MBP:stanford-corenlp-4.0.0 krishns18$ java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer
[main] INFO CoreNLP - --- StanfordCoreNLPServer#main() called ---
[main] INFO CoreNLP - Warning: cannot find edu/stanford/nlp/models/srparser/englishSR.ser.gz
[main] INFO CoreNLP - Setting default constituency parser to PCFG parser: edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz
[main] INFO CoreNLP - To use shift reduce parser download English models jar from:
[main] INFO CoreNLP - https://stanfordnlp.github.io/CoreNLP/download.html
[main] INFO CoreNLP -     Threads: 16
[main] INFO CoreNLP - Starting server...
[main] INFO CoreNLP - StanfordCoreNLPServer listening at /0:0:0:0:0:0:0:0:9000
``

In [2]:
import sys

# Add the 'utils' directory to the system path
utils_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/utils'
sys.path.insert(0, utils_path)

from text_cleaner import clean_text


In [34]:
# nlp = StanfordCoreNLP('http://localhost:9000')

In [3]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

## 1.) Load dataset

In [4]:
# Load the Excel workbook
published_file_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_1_human_annotated/all_capitals_published_quotes_IRR.xlsx'

xls = pd.ExcelFile(published_file_path)
# Loop through each sheet and print the header
for sheet_name in xls.sheet_names:
    df = pd.read_excel(published_file_path, sheet_name=sheet_name)
    print(f"'{sheet_name}':")
    print(list(df.columns))



'S18 SI':
['ID CODE', 'Essay: Why am I here?', 'Attainment', 'First Gen FINAL', 'Aspirational FINAL', 'Navigational FINAL', 'Resistance FINAL', 'Perserverance FINAL', 'Filial Piety FINAL', 'Familial FINAL', 'Community Consciousness FINAL ', 'Social FINAL', 'Spiritual FINAL']
'F18 SI':
['ID CODE', 'Essay: Why am I here?', 'Attainment', 'First Gen FINAL', 'Aspirational FINAL', 'Navigational FINAL', 'Resistance FINAL', 'Perserverance FINAL', 'Filial Piety FINAL', 'Familial FINAL', 'Community Consciousness FINAL ', 'Social FINAL', 'Spiritual FINAL']
'S19 SI':
['ID CODE', 'Essay: Why am I here?', 'Attainment', 'First Gen FINAL', 'Aspirational FINAL', 'Navigational FINAL', 'Resistance FINAL', 'Perserverance FINAL', 'Filial Piety FINAL', 'Familial FINAL', 'Community Consciousness FINAL ', 'Social FINAL', 'Spiritual FINAL']
'F19 SI':
['ID CODE', 'Essay: Why am I here?', 'Attainment', 'First Gen FINAL', 'Aspirational FINAL', 'Navigational FINAL', 'Resistance FINAL', 'Perserverance FINAL', 'Fili

In [5]:
desired_column_names = ['Alma ID', 
                        'Essay: Why I am here?', 
                        'Attainment', 
                        'First Gen', 
                        'Aspirational', 
                        'Navigational', 
                        'Resistance', 
                        'Perseverance', 
                        'Filial Piety', 
                        'Familial', 
                        'Community Consciousness',
                        'Social', 
                        'Spiritual']

In [6]:
def update_headers(file_path, output_path, desired_header):
    """
    Updates the headers of all sheets in an Excel workbook to the specified header.

    :param file_path: Path to the input Excel file.
    :param output_path: Path to the output Excel file.
    :param desired_header: List of desired headers.
    """
    # Load the Excel workbook
    xls = pd.ExcelFile(file_path)

    # Create a dictionary to store updated DataFrames
    updated_sheets = {}

    # Loop through each sheet and update the header
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
        print(f"Current headers for sheet '{sheet_name}':")
        print(list(df.iloc[0]))
        
        # Replace the existing header with the desired header
        df.columns = desired_header
        df = df[1:]  # Remove the original header row
        updated_sheets[sheet_name] = df

    # Save the updated sheets back to an Excel workbook
    with pd.ExcelWriter(output_path) as writer:
        for sheet_name, df in updated_sheets.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)


# published_file_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_1_human_annotated/all_capitals_published_quotes_IRR.xlsx'
# output_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_1_human_annotated/all_capitals_published_quotes_IRR_header_fixed.xlsx'

unpublished_file_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_1_human_annotated/all_capitals_unpublished_quotes_IRR.xlsx'
output_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_1_human_annotated/all_capitals_unpublished_quotes_IRR_header_fixed.xlsx'

update_headers(unpublished_file_path, output_path, desired_column_names)

Current headers for sheet 'Pilot':
['Alma ID', 'Essay: Why I am here?', 'Attainment', 'First Gen', 'Aspirational', 'Navigational', 'Resistance', 'Perseverance', 'Filial Piety', 'Familial', 'Community Consciousness', 'Social', 'Spiritual']
Current headers for sheet 'Spring 2018 SI 333':
['Alma ID', 'Essay: Why I am here?', 'Attainment', 'First Gen', 'Aspirational', 'Navigational', 'Resistance', 'Perseverance', 'Filial Piety', 'Familial', 'Community Consciousness', 'Social', 'Spiritual']
Current headers for sheet 'Spring 2018 - SI':
['Alma ID', 'Essay: Why I am here?', 'Attainment', 'First Gen', 'Aspirational', 'Navigational', 'Resistance', 'Perseverance', 'Filial Piety', 'Familial', 'Community Consciousness', 'Social', 'Spiritual']
Current headers for sheet 'Fall 2019 - PHYS102':
['Alma ID', 'Essay: Why I am here?', 'Attainment', 'First Gen', 'Aspirational', 'Navigational', 'Resistance', 'Perseverance', 'Filial Piety', 'Familial', 'Community Consciousness', 'Social', 'Spiritual']
Curren

In [7]:
# Read the essays and create a dataframe
df_1 = pd.concat(pd.read_excel('/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_1_human_annotated/all_capitals_published_quotes_IRR_header_fixed.xlsx', sheet_name=None),ignore_index=True,sort=False)
print(df_1.shape)

(500, 13)


In [8]:
# Read the essays and create a dataframe
df_2 = pd.concat(pd.read_excel('/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_1_human_annotated/all_capitals_unpublished_quotes_IRR_header_fixed.xlsx', sheet_name=None),ignore_index=True,sort=False)
print(df_2.shape)

(89, 13)


In [9]:
df = pd.concat([df_1, df_2])

In [10]:
df

,Alma ID,Essay: Why I am here?,Attainment,First Gen,Aspirational,Navigational,Resistance,Perseverance,Filial Piety,Familial,Community Consciousness,Social,Spiritual
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,Ever since I was little I wanted to be a docto...,0,to better one self and be able to succeed late...,0,0,0,0,0,0,0,I always wanted to be able to help people
1,S18.W2.111.01.02,I'm here because my parents came here during t...,0,0,I am also here because the formula to being su...,0,0,0,She made a lot of sacrifices for me to be here...,I'm here because my parents came here during t...,0,0,0
2,S18.W2.111.01.03,I am here because I want to better myself & my...,0,0,I am here because I want to better myself & my...,Being in this instituion will pave a way for m...,I know that as a child I never thought of edu...,0,I am here because I want to better myself & my...,0,I Hope to reach a position in which I can insp...,0,0
3,S18.W2.111.01.04,I am here at SFSU because I am a first generat...,0,I am here at SFSU because I am a first generat...,I want to help my community while I'm here and...,0,0,School has always helped me escape the hard th...,"As the oldest of 3 siblings, I want to be a ro...",0,I want to help my community while I'm here,0,0
4,S18.W2.111.01.05,I am here to help myself grow.\n- Learn and a...,0,0,- I am here to help myself grow. /%/ - Unders...,- Learn and apply my skills to progress toward...,0,0,0,0,0,Meet people who have (similar) aspirations/goa...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,F19.PHYS242.02.004.1216,My family has valued hard work. Growing up I w...,0,0,0,0,0,0,0,My family has valued hard work. Growing up I w...,0,0,0
85,F19.PHYS242.02.005.1213,I have lived in the Bay Area my whole life and...,0,0,0,0,0,0,This is something I plan on doing with my kids.,my dad told me that it’s going to be hard to g...,0,0,0
86,F19.PHYS242.02.006.1217,The values of my community have made a deep an...,0,0,0,0,0,0,0,0,The values of my community have made a deep an...,0,0
87,F19.PHYS242.02.007.1211,Well one thing my mom has taught me is to not ...,0,0,0,0,0,0,0,Well one thing my mom has taught me is to not ...,0,0,0


In [8]:
df.to_csv("batch_1_dataset_all_capitals_quotes_published_unpublished_IRR.csv", index=False)

In [11]:
df = pd.read_csv("../../../data/for_training/batch_1_processed/batch_1_dataset_all_capitals_quotes_published_unpublished_IRR.csv")

**Creating a separate dataset for Navigational**

In [12]:
df

,Alma ID,Essay: Why I am here?,Attainment,First Gen,Aspirational,Navigational,Resistance,Perseverance,Filial Piety,Familial,Community Consciousness,Social,Spiritual
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,Ever since I was little I wanted to be a docto...,0,to better one self and be able to succeed late...,0,0,0,0,0,0,0,I always wanted to be able to help people
1,S18.W2.111.01.02,I'm here because my parents came here during t...,0,0,I am also here because the formula to being su...,0,0,0,She made a lot of sacrifices for me to be here...,I'm here because my parents came here during t...,0,0,0
2,S18.W2.111.01.03,I am here because I want to better myself & my...,0,0,I am here because I want to better myself & my...,Being in this instituion will pave a way for m...,I know that as a child I never thought of edu...,0,I am here because I want to better myself & my...,0,I Hope to reach a position in which I can insp...,0,0
3,S18.W2.111.01.04,I am here at SFSU because I am a first generat...,0,I am here at SFSU because I am a first generat...,I want to help my community while I'm here and...,0,0,School has always helped me escape the hard th...,"As the oldest of 3 siblings, I want to be a ro...",0,I want to help my community while I'm here,0,0
4,S18.W2.111.01.05,I am here to help myself grow.\n- Learn and a...,0,0,- I am here to help myself grow. /%/ - Unders...,- Learn and apply my skills to progress toward...,0,0,0,0,0,Meet people who have (similar) aspirations/goa...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,F19.PHYS242.02.004.1216,My family has valued hard work. Growing up I w...,0,0,0,0,0,0,0,My family has valued hard work. Growing up I w...,0,0,0
585,F19.PHYS242.02.005.1213,I have lived in the Bay Area my whole life and...,0,0,0,0,0,0,This is something I plan on doing with my kids.,my dad told me that it’s going to be hard to g...,0,0,0
586,F19.PHYS242.02.006.1217,The values of my community have made a deep an...,0,0,0,0,0,0,0,0,The values of my community have made a deep an...,0,0
587,F19.PHYS242.02.007.1211,Well one thing my mom has taught me is to not ...,0,0,0,0,0,0,0,Well one thing my mom has taught me is to not ...,0,0,0


## 2. Create theme datasets

In [13]:
def create_theme_datasets(df, themes, sentence_level=True):
    """
    Creates separate datasets for each theme and returns them as a dictionary.

    :param df: The original DataFrame.
    :param themes: List of theme columns to create separate datasets for.
    :return: A dictionary with theme names as keys and DataFrames as values.
    """
    base_columns = ['Alma ID', 'Essay: Why I am here?']
    theme_datasets = {}

    for theme in themes:
        # Select the columns for the theme dataset
        theme_df = df[base_columns + [theme]]

        if sentence_level:
        # Filter out rows where the theme value is 0
            theme_df = theme_df[theme_df[theme] != 0]
            theme_df = theme_df[theme_df[theme] != "0"]
        
        # Add the DataFrame to the dictionary
        theme_datasets[theme] = theme_df

    return theme_datasets

theme_names = desired_column_names[2:]

# Create the theme datasets
sentence_level_theme_datasets = create_theme_datasets(df, theme_names, True)
essay_level_theme_datasets = create_theme_datasets(df, theme_names, False)

In [17]:
sentence_level_theme_datasets["Resistance"]

,Alma ID,Essay: Why I am here?,Resistance
2,S18.W2.111.01.03,I am here because I want to better myself & my...,I know that as a child I never thought of edu...
20,S18.W2.111.01.22,I am here because I care about my community an...,I am here in the science community because I f...
30,F18.SCI111.03.09,I am here because I am dedicated to my succes...,I’m here for all the little colored girls who ...
52,F18.SI.111.04.07,I am here because I am a first generation coll...,I want to show all the minorities that you can...
69,S19.SCI111.01.10,I am here because I always wanted to be involv...,"I am an African American male, and there is n..."
70,S19.SCI111.02.01,I am here in this world to make an impact. I a...,Not only am I making an impact on the medical ...
182,F19.PHYS122.01.02,I am here to take this physics class so I can ...,Iíve seen what good and bad doctors have done ...
190,F19.PHYS122.01.12,I am here for a list of reasons. Reasons that ...,"Firstly, I am here to challenge myself and mak..."
192,F19.PHYS122.01.14,Why I am here is because I want to fulfill my ...,I was born in Afghanistan in a house during th...
214,F19.PHYS122.08.06,Iím in college because I know that I can make ...,I have a drive to do better in a world that co...


In [156]:
essay_level_theme_datasets["Navigational"]

,Alma ID,Essay: Why I am here?,Navigational
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,0
1,S18.W2.111.01.02,I'm here because my parents came here during t...,0
2,S18.W2.111.01.03,I am here because I want to better myself & my...,Being in this instituion will pave a way for m...
3,S18.W2.111.01.04,I am here at SFSU because I am a first generat...,0
4,S18.W2.111.01.05,I am here to help myself grow.\n- Learn and a...,- Learn and apply my skills to progress toward...
...,...,...,...
584,F19.PHYS242.02.004.1216,My family has valued hard work. Growing up I w...,0
585,F19.PHYS242.02.005.1213,I have lived in the Bay Area my whole life and...,0
586,F19.PHYS242.02.006.1217,The values of my community have made a deep an...,0
587,F19.PHYS242.02.007.1211,Well one thing my mom has taught me is to not ...,0


In [22]:
essay_level_theme_datasets["Navigational"]

,Alma ID,Essay: Why I am here?,Navigational
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,0
1,S18.W2.111.01.02,I'm here because my parents came here during t...,0
2,S18.W2.111.01.03,I am here because I want to better myself & my...,Being in this instituion will pave a way for m...
3,S18.W2.111.01.04,I am here at SFSU because I am a first generat...,0
4,S18.W2.111.01.05,I am here to help myself grow.\n- Learn and a...,- Learn and apply my skills to progress toward...
...,...,...,...
584,F19.PHYS242.02.004.1216,My family has valued hard work. Growing up I w...,0
585,F19.PHYS242.02.005.1213,I have lived in the Bay Area my whole life and...,0
586,F19.PHYS242.02.006.1217,The values of my community have made a deep an...,0
587,F19.PHYS242.02.007.1211,Well one thing my mom has taught me is to not ...,0


In [47]:
navigational_sentence_df = sentence_level_theme_datasets["Navigational"]

**Data Preparation**  
1. Get noun and adjective

In [20]:
def get_noun_adj(phrase):
    noun_adj = []
    for tag in pos_tag(phrase.split()):
        if tag[1] in ['NN','NNP','NNS', 'NNPS','JJ', 'JJR', 'JJS']:
            noun_adj.append(tag[0])
    return  noun_adj

In [21]:
## to replace get_ner??
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score

def jaccard_similarity(str1, str2):
    vect = CountVectorizer().fit_transform([str1, str2])
    vectors = vect.toarray()
    return jaccard_score(vectors[0], vectors[1], average='weighted')


3. Get sentence label
    1. Look for exact match
    2. Check for NER in phrase (title/profession/scientific fields) and check if those exist in the sentence
    3. Get the nouns and adjectives and check if those exist in the sentence

In [22]:
text = "I‚Äôm here because I want to become a dentist. How about you? Let's do this!"
cleaned_text = clean_text(text)
cleaned_text

"Im here because I want to become a dentist. How about you? Let's do this!"

In [23]:
## updated with jaccard sim and removed get_ner

def get_sentence_label(essay_sentence, phrase): 
    # Escape special regex characters in the phrase but keep / and - unescaped
    escaped_phrase = re.escape(phrase).replace(r'\/', '/').replace(r'\-', '-')
    # Create a regex pattern to find the phrase within the essay sentence
    match = re.search(r'.*%s.*' % escaped_phrase, essay_sentence, re.IGNORECASE)
    if match:
        return 1
    else:        
        similarity = jaccard_similarity(essay_sentence, phrase)
        # print(similarity)
        if similarity > 0.5:  # Adjust the threshold as needed
            return 1
        else:
            # 3. Else get the nouns and adjectives and check if those present in the sentence
            noun_adj_phrase = get_noun_adj(phrase)
            all_pos_found = None
            for pos in noun_adj_phrase:
                if pos not in essay_sentence:
                    all_pos_found = 0
                    break
                else:
                    all_pos_found = 1
            if all_pos_found == 1:
                return 1
            else:
                return 0


In [24]:
get_sentence_label("a one hour airplane trip back home, 7 hour drive",'I am in college specifically to pursue my one and only career which is to be a veterinarian')

0

In [25]:
from custom_tokenizer import custom_sent_tokenizer

In [26]:

text = "Dr. Smith is a veterinarian. P.A. Jones assists him."
sentences = custom_sent_tokenizer(text)
print(sentences)

['Dr. Smith is a veterinarian.', 'P.A. Jones assists him.']


In [27]:
sent = "‚Ä†i am at san francisco state university to get a bachelors degree in civil engineering "
sentences = clean_text(sent)
sentences = custom_sent_tokenizer(sentences)

print(sentences)



['i am at san francisco state university to get a bachelors degree in civil engineering']


In [28]:
clean_text(sent)

'i am at san francisco state university to get a bachelors degree in civil engineering'

## Looking at example 1, index 0

In [60]:
# entire essay
navigational_sentence_df["Essay: Why I am here?"][2]

'I am here because I want to better myself & my family, not only financially but in health. Being in this instituion will pave a way for me to become a professional. I know that as a child I never thought of education and a career for someone who is undocumented. I Hope to reach a position in which I can inspire and prove to all of the "immigrants" in the USA that achieving and surpassing struggle is possible. Also, statistics have shown that people with higher education and wealth tend to be healthier. With my career & diploma I can improve my health & the health of my family. '

In [59]:
navigational_sentence_df

,Alma ID,Essay: Why I am here?,Navigational
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,0
1,S18.W2.111.01.02,I'm here because my parents came here during t...,0
2,S18.W2.111.01.03,I am here because I want to better myself & my...,Being in this instituion will pave a way for m...
3,S18.W2.111.01.04,I am here at SFSU because I am a first generat...,0
4,S18.W2.111.01.05,I am here to help myself grow.\n- Learn and a...,- Learn and apply my skills to progress toward...
...,...,...,...
584,F19.PHYS242.02.004.1216,My family has valued hard work. Growing up I w...,0
585,F19.PHYS242.02.005.1213,I have lived in the Bay Area my whole life and...,0
586,F19.PHYS242.02.006.1217,The values of my community have made a deep an...,0
587,F19.PHYS242.02.007.1211,Well one thing my mom has taught me is to not ...,0


In [180]:
essay_level_theme_datasets["Navigational"]

,Alma ID,Essay: Why I am here?,Navigational
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,0
1,S18.W2.111.01.02,I'm here because my parents came here during t...,0
2,S18.W2.111.01.03,I am here because I want to better myself & my...,Being in this instituion will pave a way for m...
3,S18.W2.111.01.04,I am here at SFSU because I am a first generat...,0
4,S18.W2.111.01.05,I am here to help myself grow.\n- Learn and a...,- Learn and apply my skills to progress toward...
...,...,...,...
584,F19.PHYS242.02.004.1216,My family has valued hard work. Growing up I w...,0
585,F19.PHYS242.02.005.1213,I have lived in the Bay Area my whole life and...,0
586,F19.PHYS242.02.006.1217,The values of my community have made a deep an...,0
587,F19.PHYS242.02.007.1211,Well one thing my mom has taught me is to not ...,0


In [31]:
#sentencep positive for attainment
attainment_df["Attainment - FINAL"][0]

'I am here to get my BS in physiology so I can get into medical school.'

## 3. Creating sentence level phrases

In [29]:
sent = "i feel like i wasn‚Äôt created to be a cardiothoracic surgeon."

clean_text(sent)

'i feel like i wasnt created to be a cardiothoracic surgeon.'

In [77]:
navigational_sentence_df[navigational_sentence_df["Alma ID"] == "S19.SCI111.02.09"]["Navigational"].to_list()[0]

' I’ve had a history of struggling in my math classes. Since physics is basically another math class, I knew I would need to take extra steps to ensure my success. /-/ When I heard the option to enroll in a supplemental course for physics, I knew I had to enroll for the extra help.'

In [75]:
navigational_sentence_df[navigational_sentence_df["Alma ID"] == "S19.SCI111.02.09"]["Essay: Why I am here?"].to_list()[0]

'I decided to enroll into the physics supplemental course because I’ve had a history of struggling in my math classes. Since physics is basically another math class, I knew I would need to take extra steps to ensure my success. In the past I’ve had a packed schedule with classes and work, but this semester I allowed myself more room for school by cutting back my work hours. I originally planned on letting myself have time to attend office hours, but I realized that wouldn’t be enough.\nWhen I heard the option to enroll in a supplemental course for physics, I knew I had to enroll for the extra help. Not only would I have more practice with physics problems, but I would be able to connect more with my fellow classmates. I figured that if I felt like I created a little community with the supplemental course, I would feel more comfortable with physics problems. With a community I would feel comfortable enough to ask questions without having to worry about sounding unintelligent.\nAnother b

'I decided to enroll into the physics supplemental course because I’ve had a history of struggling in my math classes. Since physics is basically another math class, I knew I would need to take extra steps to ensure my success. In the past I’ve had a packed schedule with classes and work, but this semester I allowed myself more room for school by cutting back my work hours. I originally planned on letting myself have time to attend office hours, but I realized that wouldn’t be enough.\nWhen I heard the option to enroll in a supplemental course for physics, I knew I had to enroll for the extra help. Not only would I have more practice with physics problems, but I would be able to connect more with my fellow classmates. I figured that if I felt like I created a little community with the supplemental course, I would feel more comfortable with physics problems. With a community I would feel comfortable enough to ask questions without having to worry about sounding unintelligent.\nAnother big reason for enrolling was when I heard I wouldn’t have homework. I will have the opportunity to have the extra practice with physics problems without the stress of having to worry about my grade. I will be able to do the problems at my own pace and truly understand the steps to get the solution'

In [81]:
ex = 'I decided to enroll into the physics supplemental course because I’ve had a history of struggling in my math classes. Since physics is basically another math class, I knew I would need to take extra steps to ensure my success. In the past I’ve had a packed schedule with classes and work, but this semester I allowed myself more room for school by cutting back my work hours. I originally planned on letting myself have time to attend office hours, but I realized that wouldn’t be enough.\nWhen I heard the option to enroll in a supplemental course for physics, I knew I had to enroll for the extra help. Not only would I have more practice with physics problems, but I would be able to connect more with my fellow classmates. I figured that if I felt like I created a little community with the supplemental course, I would feel more comfortable with physics problems. With a community I would feel comfortable enough to ask questions without having to worry about sounding unintelligent.\nAnother big reason for enrolling was when I heard I wouldn’t have homework. I will have the opportunity to have the extra practice with physics problems without the stress of having to worry about my grade. I will be able to do the problems at my own pace and truly understand the steps to get the solution'
annotation = ' I’ve had a history of struggling in my math classes. Since physics is basically another math class /%/ I knew I would need to take extra steps to ensure my success. /-/ When I heard the option to enroll in a supplemental course for physics. /-/ I knew I had to enroll for the extra help.'

annotation_list = re.split(r'/%/', annotation)
annotation_list = [clean_text(text) for text in annotation_list]
annotation_list

['Ive had a history of struggling in my math classes. Since physics is basically another math class',
 'I knew I would need to take extra steps to ensure my success. When I heard the option to enroll in a supplemental course for physics. I knew I had to enroll for the extra help.']

In [63]:
annotation = "- I am here to help myself grow. /%/  - Understand my purpose. /%/  Improve /%/ - Learn and apply my skills to progress towards my career goal."

annotation_list = re.split(r'/%/', annotation)
annotation_list = [clean_text(text) for text in annotation_list]
annotation_list

['I am here to help myself grow.',
 'Understand my purpose.',
 'Improve',
 'Learn and apply my skills to progress towards my career goal.']

In [31]:
def process_sentences_for_theme(theme_df, theme_column):
    theme_sentence_df = pd.DataFrame(columns=['sentence', 'label', 'phrase'])
    
    for index, val in theme_df.iterrows():
        text = val['Essay: Why I am here?'].rstrip('\n\r')
        
        # Split the paragraphs into sentences using sent_tokenize from nltk
        temp_text = custom_sent_tokenizer(text)
        
        # Annotation
        annotation = val[theme_column]
         
        # Annotation list
        annotation_list = re.split(r'/%/', annotation)
        annotation_strip = [clean_text(annotation) for annotation in annotation_list]
        essay_strip = [x for x in temp_text if x != '']
        essay_strip = [clean_text(essay) for essay in essay_strip]
        
        # Remove extra spaces before the sentences
        annotation_strip = [y.lstrip(" ") for y in annotation_strip]
        essay_strip = [x.lstrip(" ") for x in essay_strip]
        
        # Annotate the essay on sentence level
        for sentence in essay_strip:
            sentence = clean_text(sentence)
            if not re.search(r'\w', sentence):
                continue
            if len(sentence.split(' ')) < 3:
                continue
            
            label = 0
            for annotation_phrase in annotation_strip:
                if len(annotation_phrase.split()) > 2:
                    label = get_sentence_label(sentence, annotation_phrase)
                    if label == 1:
                        break
            
            # Create a new DataFrame with the row you want to add
            new_row = pd.DataFrame([{'sentence': sentence.lower(), 'label': label, 'phrase': annotation_strip}])
            
            # Concatenate the new row to the existing DataFrame
            theme_sentence_df = pd.concat([theme_sentence_df, new_row], ignore_index=True)
    
    return theme_sentence_df

In [189]:
processed_essay_level_datasets = process_essays_for_theme(essay_level_theme_datasets["Navigational"], "Navigational")
processed_essay_level_datasets

,essay,label,phrase
0,[since i am the first one go to college in my ...,"Series([], Name: label, dtype: object)",[0]


In [ ]:
processed_essay_level_datasets

In [190]:
essay_level_theme_datasets["Navigational"]

,Alma ID,Essay: Why I am here?,Navigational
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,0
1,S18.W2.111.01.02,I'm here because my parents came here during t...,0
2,S18.W2.111.01.03,I am here because I want to better myself & my...,Being in this instituion will pave a way for m...
3,S18.W2.111.01.04,I am here at SFSU because I am a first generat...,0
4,S18.W2.111.01.05,I am here to help myself grow.\n- Learn and a...,- Learn and apply my skills to progress toward...
...,...,...,...
584,F19.PHYS242.02.004.1216,My family has valued hard work. Growing up I w...,0
585,F19.PHYS242.02.005.1213,I have lived in the Bay Area my whole life and...,0
586,F19.PHYS242.02.006.1217,The values of my community have made a deep an...,0
587,F19.PHYS242.02.007.1211,Well one thing my mom has taught me is to not ...,0


In [192]:
processed_theme_datasets["Navigational"]

,sentence,label,phrase
0,i am here because i want to better myself my f...,0,[Being in this instituion will pave a way for ...
1,being in this instituion will pave a way for m...,1,[Being in this instituion will pave a way for ...
2,i know that as a child i never thought of educ...,0,[Being in this instituion will pave a way for ...
3,i hope to reach a position in which i can insp...,0,[Being in this instituion will pave a way for ...
4,"also, statistics have shown that people with h...",0,[Being in this instituion will pave a way for ...
...,...,...,...
3191,"i want to better myself, improve myself in eve...",0,[I am here to fulfill my mechanical engineerin...
3192,i have no problems applying myself when it com...,0,[I am here to fulfill my mechanical engineerin...
3193,grow mentally so that i can apply more physica...,0,[I am here to fulfill my mechanical engineerin...
3194,use what i learn and apply it in whatever care...,0,[I am here to fulfill my mechanical engineerin...


In [211]:
def process_essays_for_theme(theme_df, theme_column):
    theme_essay_df = pd.DataFrame(columns=['essay', 'label', 'phrase'])

    essay_from_df = theme_df["Essay: Why I am here?"]
    essay_list = [clean_text(essay) for essay in essay_from_df]

    label_list = [1 if value != "0" and value != 0 else 0 for value in theme_df[theme_column]]
    annotation_from_df = theme_df[theme_column]
    annotation_list = []
    
    for annotation in annotation_from_df:
        annotation_instances = re.split(r'/%/', annotation)
        annotation_instances = [clean_text(a) for a in annotation_instances]
        annotation_list.append(annotation_instances)

    theme_essay_df["essay"] = essay_list
    theme_essay_df["label"] = label_list
    theme_essay_df["phrase"] = annotation_list

    return theme_essay_df

In [212]:
processed_essay_level_datasets = process_essays_for_theme(essay_level_theme_datasets["Navigational"], "Navigational")
processed_essay_level_datasets

,essay,label,phrase
0,Why am I here? Well why does anyone pursue a h...,0,[0]
1,I'm here because my parents came here during t...,0,[0]
2,I am here because I want to better myself my f...,1,[Being in this instituion will pave a way for ...
3,I am here at SFSU because I am a first generat...,0,[0]
4,I am here to help myself grow. Learn and apply...,1,[Learn and apply my skills to progress towards...
...,...,...,...
584,My family has valued hard work. Growing up I w...,0,[0]
585,I have lived in the Bay Area my whole life and...,0,[0]
586,The values of my community have made a deep an...,0,[0]
587,Well one thing my mom has taught me is to not ...,0,[0]


In [32]:
essay_level_theme_names = sentence_level_theme_names
essay_level_theme_names

NameError: name 'sentence_level_theme_names' is not defined

In [218]:
processed_essay_theme_datasets = {}

for i in sentence_level_theme_names:
    print("Working on " + i)
    processed_name = f"{i}_processed_essay_df"
    processed_essay_theme_datasets[i] = process_essays_for_theme(essay_level_theme_datasets[i], i)

Working on Navigational
Working on Perseverance
Working on Resistance


In [220]:
processed_essay_theme_datasets["Perseverance"]

,essay,label,phrase
0,Why am I here? Well why does anyone pursue a h...,0,[0]
1,I'm here because my parents came here during t...,0,[0]
2,I am here because I want to better myself my f...,0,[0]
3,I am here at SFSU because I am a first generat...,1,[School has always helped me escape the hard t...
4,I am here to help myself grow. Learn and apply...,0,[0]
...,...,...,...
584,My family has valued hard work. Growing up I w...,0,[0]
585,I have lived in the Bay Area my whole life and...,0,[0]
586,The values of my community have made a deep an...,0,[0]
587,Well one thing my mom has taught me is to not ...,0,[0]


In [213]:
essay_level_theme_datasets["Navigational"]

,Alma ID,Essay: Why I am here?,Navigational
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,0
1,S18.W2.111.01.02,I'm here because my parents came here during t...,0
2,S18.W2.111.01.03,I am here because I want to better myself & my...,Being in this instituion will pave a way for m...
3,S18.W2.111.01.04,I am here at SFSU because I am a first generat...,0
4,S18.W2.111.01.05,I am here to help myself grow.\n- Learn and a...,- Learn and apply my skills to progress toward...
...,...,...,...
584,F19.PHYS242.02.004.1216,My family has valued hard work. Growing up I w...,0
585,F19.PHYS242.02.005.1213,I have lived in the Bay Area my whole life and...,0
586,F19.PHYS242.02.006.1217,The values of my community have made a deep an...,0
587,F19.PHYS242.02.007.1211,Well one thing my mom has taught me is to not ...,0


In [83]:
sentence_level_theme_names = ["Navigational", "Perseverance", "Resistance"]

In [96]:
sentence_level_theme_names = ["Attainment", "Aspirational", "Familial", "Filial Piety", "First Gen", "Community Consciousness", "Spiritual", "Social"]

In [85]:
sentence_level_theme_names

['Navigational', 'Perseverance', 'Resistance']

In [86]:
sentence_level_theme_datasets

NameError: name 'theme_datasets' is not defined

In [162]:
navigational_processed_sentence_df_2 = process_sentences_for_theme(sentence_level_theme_datasets["Resistance"], "Resistance")
navigational_processed_sentence_df_2

,sentence,label,phrase
0,i am here because i want to better myself my f...,0,[I know that as a child I never thought of edu...
1,being in this instituion will pave a way for m...,0,[I know that as a child I never thought of edu...
2,i know that as a child i never thought of educ...,1,[I know that as a child I never thought of edu...
3,i hope to reach a position in which i can insp...,1,[I know that as a child I never thought of edu...
4,"also, statistics have shown that people with h...",0,[I know that as a child I never thought of edu...
...,...,...,...
152,im currently lost as to how it is that i want ...,0,"[I want to represent myself, my gender, my sex..."
153,i am here because of the way that i have grown...,0,[I am here to disprove stereotypes.]
154,im here because of all the sacrifices that bot...,0,[I am here to disprove stereotypes.]
155,im here to disapprove stereotypes im here for ...,0,[I am here to disprove stereotypes.]


In [120]:
navigational_processed_sentence_df_2[navigational_processed_sentence_df_2["label"] == 1]

16

In [224]:
attainment_processed_sentence_df["label"].value_counts()

label
0    1437
1     304
Name: count, dtype: int64

In [223]:
attainment_processed_sentence_df_2["label"].value_counts()

label
0    1407
1     261
Name: count, dtype: int64

In [221]:
attainment_processed_sentence_df_2
attainment_processed_sentence_df_2.to_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/atn_sentence_level_batch_1_jaccard.csv", index=False)

In [91]:
sentence_level_theme_names

['Navigational', 'Perseverance', 'Resistance']

NameError: name 'processed_theme_datasets' is not defined

In [165]:
sentence_level_theme_datasets["Resistance"]

,Alma ID,Essay: Why I am here?,Resistance
2,S18.W2.111.01.03,I am here because I want to better myself & my...,I know that as a child I never thought of edu...
20,S18.W2.111.01.22,I am here because I care about my community an...,I am here in the science community because I f...
30,F18.SCI111.03.09,I am here because I am dedicated to my succes...,I’m here for all the little colored girls who ...
52,F18.SI.111.04.07,I am here because I am a first generation coll...,I want to show all the minorities that you can...
69,S19.SCI111.01.10,I am here because I always wanted to be involv...,"I am an African American male, and there is n..."
70,S19.SCI111.02.01,I am here in this world to make an impact. I a...,Not only am I making an impact on the medical ...
182,F19.PHYS122.01.02,I am here to take this physics class so I can ...,Iíve seen what good and bad doctors have done ...
190,F19.PHYS122.01.12,I am here for a list of reasons. Reasons that ...,"Firstly, I am here to challenge myself and mak..."
192,F19.PHYS122.01.14,Why I am here is because I want to fulfill my ...,I was born in Afghanistan in a house during th...
214,F19.PHYS122.08.06,Iím in college because I know that I can make ...,I have a drive to do better in a world that co...


In [33]:
sentence_level_theme_names = ["Navigational", "Perseverance", "Resistance"]

In [34]:
processed_theme_datasets = {}

for i in sentence_level_theme_names: #skipping Attainment because we already did it
    print("Working on " + i)
    processed_name = f"{i}_processed_sentence_df"
    processed_theme_datasets[i] = process_sentences_for_theme(sentence_level_theme_datasets[i], i)

Working on Navigational
Working on Perseverance
Working on Resistance


In [96]:
sentence_level_theme_datasets["Navigational"]["Essay: Why I am here?"][565]

"I am here to learn the basics of physics through experiments, also to graduate as the first one in my family from a college university. This should help me in the future because it is for my major requirement for kinesiology. I am also enrolled in the physics lecture. Physics is in everyday life so this class will come in handy. Currently I'm learning conversions which I am bad at. I wish to improve my math skills and conversion skills. This class will prepare me for upper division classes, even though its my last physics class, it should come in handy in everyday problem solving. For example if I ever need to calculate how much gas I need to pump in my car. I can calculate it with money, gallons, and miles. So this class will enhance my problem solving skills right now in life, and later in life. I am here to learn.†"

In [299]:
from utils.text_cleaner import clean_text



In [35]:
for k,v in processed_theme_datasets.items():
    print(k, len(v))

Navigational 3196
Perseverance 1047
Resistance 157


: 

In [170]:
processed_theme_datasets["Navigational"]

,sentence,label,phrase
0,i am here because i want to better myself my f...,0,[Being in this instituion will pave a way for ...
1,being in this instituion will pave a way for m...,1,[Being in this instituion will pave a way for ...
2,i know that as a child i never thought of educ...,0,[Being in this instituion will pave a way for ...
3,i hope to reach a position in which i can insp...,0,[Being in this instituion will pave a way for ...
4,"also, statistics have shown that people with h...",0,[Being in this instituion will pave a way for ...
...,...,...,...
3191,"i want to better myself, improve myself in eve...",0,[I am here to fulfill my mechanical engineerin...
3192,i have no problems applying myself when it com...,0,[I am here to fulfill my mechanical engineerin...
3193,grow mentally so that i can apply more physica...,0,[I am here to fulfill my mechanical engineerin...
3194,use what i learn and apply it in whatever care...,0,[I am here to fulfill my mechanical engineerin...


In [230]:
aspirational_processed_sentence_df = processed_theme_datasets["Aspirational"]
aspirational_processed_sentence_df.to_csv("")

,sentence,label,phrase
0,why am i here?,0,[to better one self and be able to succeed lat...
1,well why does anyone pursue a higher education?,0,[to better one self and be able to succeed lat...
2,to better one self and be able to succeed late...,1,[to better one self and be able to succeed lat...
3,ever since i was little i wanted to be a docto...,0,[to better one self and be able to succeed lat...
4,i always wanted to be able to help people and ...,1,[to better one self and be able to succeed lat...
...,...,...,...
3031,"i want to better myself, improve myself in eve...",1,[I want to be able to apply my knowledge to my...
3032,i have no problems applying myself when it com...,0,[I want to be able to apply my knowledge to my...
3033,grow mentally so that i can apply more physica...,0,[I want to be able to apply my knowledge to my...
3034,use what i learn and apply it in whatever care...,0,[I want to be able to apply my knowledge to my...


In [225]:
for k,v in processed_essay_theme_datasets.items():
    print(k, len(v))
    v.to_csv(f"/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/single_theme_essay_level/{k}_essay_level_batch_1_jaccard.csv", index=False)

Navigational 589
Perseverance 589
Resistance 589


In [172]:
for k,v in processed_theme_datasets.items():
    print(k, len(v))
    v.to_csv(f"/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/single_theme_using_jaccard_method/{k}_sentence_level_batch_1_jaccard.csv", index=False)

Navigational 3196
Perseverance 1047
Resistance 157


In [240]:
merged_resistance_essay_df = pd.concat([processed_essay_theme_datasets["Perseverance"], processed_essay_theme_datasets["Resistance"]])
merged_resistance_essay_df.to_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/essay_level/merged_themes_essay_level/merged_Resistance_essay_level_batch_1.csv", index=False)

In [243]:
pd.read_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/essay_level/merged_themes_essay_level/merged_Resistance_essay_level_batch_1.csv")


,essay,label,phrase
0,Why am I here? Well why does anyone pursue a h...,0,['0']
1,I'm here because my parents came here during t...,0,['0']
2,I am here because I want to better myself my f...,0,['0']
3,I am here at SFSU because I am a first generat...,1,['School has always helped me escape the hard ...
4,I am here to help myself grow. Learn and apply...,0,['0']
...,...,...,...
1173,My family has valued hard work. Growing up I w...,0,['0']
1174,I have lived in the Bay Area my whole life and...,0,['0']
1175,The values of my community have made a deep an...,0,['0']
1176,Well one thing my mom has taught me is to not ...,0,['0']


## 3. Merging similar themes 

1. Attainment and aspirational
2. Familial, filial piety, and first-generation
3. Perserverance and resistance (essay level)
4. Community consciousness, spiritual, and social
5. Navigational (essay level)


In [176]:
processed_theme_datasets["Navigational"]

,sentence,label,phrase
0,i am here because i want to better myself my f...,0,[Being in this instituion will pave a way for ...
1,being in this instituion will pave a way for m...,1,[Being in this instituion will pave a way for ...
2,i know that as a child i never thought of educ...,0,[Being in this instituion will pave a way for ...
3,i hope to reach a position in which i can insp...,0,[Being in this instituion will pave a way for ...
4,"also, statistics have shown that people with h...",0,[Being in this instituion will pave a way for ...
...,...,...,...
3191,"i want to better myself, improve myself in eve...",0,[I am here to fulfill my mechanical engineerin...
3192,i have no problems applying myself when it com...,0,[I am here to fulfill my mechanical engineerin...
3193,grow mentally so that i can apply more physica...,0,[I am here to fulfill my mechanical engineerin...
3194,use what i learn and apply it in whatever care...,0,[I am here to fulfill my mechanical engineerin...


In [323]:
[processed_theme_datasets[i] for i in merged_themes["Aspirational"]]

,sentence,label,phrase
0,why am i here?,0,[to better one self and be able to succeed lat...
1,well why does anyone pursue a higher education?,0,[to better one self and be able to succeed lat...
2,to better one self and be able to succeed late...,1,[to better one self and be able to succeed lat...
3,ever since i was little i wanted to be a docto...,0,[to better one self and be able to succeed lat...
4,i always wanted to be able to help people and ...,0,[to better one self and be able to succeed lat...
...,...,...,...
3031,"i want to better myself, improve myself in eve...",0,[I want to be able to apply my knowledge to my...
3032,i have no problems applying myself when it com...,0,[I want to be able to apply my knowledge to my...
3033,grow mentally so that i can apply more physica...,0,[I want to be able to apply my knowledge to my...
3034,use what i learn and apply it in whatever care...,0,[I want to be able to apply my knowledge to my...


In [229]:
merged_resistance_essay_df

,essay,label,phrase
0,Why am I here? Well why does anyone pursue a h...,0,[0]
1,I'm here because my parents came here during t...,0,[0]
2,I am here because I want to better myself my f...,0,[0]
3,I am here at SFSU because I am a first generat...,1,[School has always helped me escape the hard t...
4,I am here to help myself grow. Learn and apply...,0,[0]
...,...,...,...
584,My family has valued hard work. Growing up I w...,0,[0]
585,I have lived in the Bay Area my whole life and...,0,[0]
586,The values of my community have made a deep an...,0,[0]
587,Well one thing my mom has taught me is to not ...,0,[0]


In [173]:


# merged_themes = {"Aspirational": ["Attainment", "Aspirational"],
#                  "Familial": ["Familial", "Filial Piety", "First Gen"],
#                  "Social": ["Community Consciousness", "Spiritual", "Social"]}


# merged_aspirational_df = pd.concat([processed_theme_datasets[i] for i in merged_themes["Aspirational"]])
# merged_familial_df = pd.concat([processed_theme_datasets[i] for i in merged_themes["Familial"]])
# merged_social_df = pd.concat([processed_theme_datasets[i] for i in merged_themes["Social"]])

merged_themes = {"Resistance": ["Resistance", "Perseverance"]}
merged_resistance_df = pd.concat([processed_theme_datasets[i] for i in merged_themes["Resistance"]])

In [328]:
len(attainment_processed_sentence_df[attainment_processed_sentence_df["label"] == 1])

304

In [247]:
processed_essay_theme_datasets["Navigational"]

,essay,label,phrase
0,Why am I here? Well why does anyone pursue a h...,0,[0]
1,I'm here because my parents came here during t...,0,[0]
2,I am here because I want to better myself my f...,1,[Being in this instituion will pave a way for ...
3,I am here at SFSU because I am a first generat...,0,[0]
4,I am here to help myself grow. Learn and apply...,1,[Learn and apply my skills to progress towards...
...,...,...,...
584,My family has valued hard work. Growing up I w...,0,[0]
585,I have lived in the Bay Area my whole life and...,0,[0]
586,The values of my community have made a deep an...,0,[0]
587,Well one thing my mom has taught me is to not ...,0,[0]


In [249]:
merged_essay_level_datasets = {"Resistance": merged_resistance_essay_df,
"Navigational": processed_essay_theme_datasets["Navigational"]
                                  }

for k, v in merged_essay_level_datasets.items():
    print(k, v["label"].value_counts(), f"percentage of themed essays: {round(len(v[v['label'] == 1]) / len(v) * 100, 2)}%")


Resistance label
0    1031
1     147
Name: count, dtype: int64 percentage of themed essays: 12.48%
Navigational label
1    401
0    188
Name: count, dtype: int64 percentage of themed essays: 68.08%


In [177]:
merged_sentence_level_datasets = {"Resistance": merged_resistance_df,
"Navigational": processed_theme_datasets["Navigational"]
                                  }

for k, v in merged_sentence_level_datasets.items():
    print(k, v["label"].value_counts(), f"percentage of themed essays: {round(len(v[v['label'] == 1]) / len(v) * 100, 2)}%")


Resistance label
0    1117
1      87
Name: count, dtype: int64 percentage of themed essays: 7.23%
Navigational label
0    2912
1     284
Name: count, dtype: int64 percentage of themed essays: 8.89%


In [178]:
for k,v in merged_sentence_level_datasets.items():
    v.to_csv(f"/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/merged_themes_using_jaccard_method/merged_{k}_sentence_level_batch_1_jaccard.csv", index=False)

In [118]:
processed_theme_datasets.keys()

dict_keys(['Aspirational_processed_sentence_df', 'Familial_processed_sentence_df', 'Filial Piety_processed_sentence_df', 'First Gen_processed_sentence_df', 'Community Consciousness_processed_sentence_df', 'Spiritual_processed_sentence_df', 'Social_processed_sentence_df'])

In [119]:
atn_asp_df = pd.concat([attainment_processed_sentence_df, aspirational_processed_sentence_df])
atn_asp_df

,sentence,label,phrase
0,why am i here,0,[Ever since I was little I wanted to be a doct...
1,well why does anyone pursue a higher education,0,[Ever since I was little I wanted to be a doct...
2,to better one self and be able to succeed late...,0,[Ever since I was little I wanted to be a doct...
3,ever since i was little i wanted to be a docto...,1,[Ever since I was little I wanted to be a doct...
4,i always wanted to be able to help people and ...,0,[Ever since I was little I wanted to be a doct...
...,...,...,...
3151,"i want to better myself, improve myself in eve...",1,[I want to be able to apply my knowledge to my...
3152,i have no problems applying myself when it com...,0,[I want to be able to apply my knowledge to my...
3153,grow mentally so that i can apply more physica...,0,[I want to be able to apply my knowledge to my...
3154,use what i learn and apply it in whatever care...,0,[I want to be able to apply my knowledge to my...


/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti


In [127]:
atn_asp_df.to_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/atn_asp_sentence_level_batch_1.csv", index=False)


In [56]:
attainment_sentence_df = sentence_level_theme_datasets["Attainment"]
attainment_sentence_df

,Alma ID,Essay: Why I am here?,Attainment
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,Ever since I was little I wanted to be a docto...
6,S18.W2.111.01.07,I am here to better myself and hopefully give ...,I am in this class because I want to pass phys...
8,S18.W2.111.01.09,I am here in this class because I want a littl...,I am majoring in bio to get to my career goal ...
11,S18.W2.111.01.12,I am here at SFSU for a better future. A futur...,I aspire to become a doctor /%/ To achieve goa...
16,S18.W2.111.01.17,I'm here because I want to improve my understa...,"In the end, the goal is to achieve an A for my..."
...,...,...,...
68,F19.PHYS102.02.08,Why am I here? If you're asking me why am I on...,I would tell you that I am chasing a degree. I...
72,F19.PHYS242.05.01,I am here at SFSU for a multitude of reasons. ...,My second is to receive a degree in Engineerin...
73,F19.PHYS242.05.02,I am here because I want to make my mom proud....,I want to graduate
75,F19.PHYS242.05.04,Why am I here? †I am at San Francisco State Un...,I am at San Francisco State University to get ...


In [78]:
attainment_processed_sentence_df = process_sentences_for_theme(attainment_sentence_df, "Attainment")

KeyboardInterrupt: 

In [94]:
attainment_processed_sentence_df[14:]

,sentence,label,phrase
14,i knew this step was going to be an important ...,0,[I am majoring in bio to get to my career goal...
15,i am here at sfsu for a better future,0,"[I aspire to become a doctor , To achieve goal..."
16,a future where i don't have to struggle at the...,0,"[I aspire to become a doctor , To achieve goal..."
17,to achieve goals such as open up a homeless sh...,1,"[I aspire to become a doctor , To achieve goal..."
18,i aspire to become a doctor because i know tha...,1,"[I aspire to become a doctor , To achieve goal..."
...,...,...,...
1736,†by the time i got here at sfsu i decided to j...,0,[I am at San Francisco State University to get...
1737,†right now i am slightly considering switching...,0,[I am at San Francisco State University to get...
1738,one of the main reasons why i came to californ...,1,[One of the main reasons why I came to Califor...
1739,i move to san francisco because i got accepted...,0,[One of the main reasons why I came to Califor...


In [93]:
attainment_processed_sentence_df["sentence"][18]

'i aspire to become a doctor because i know that not only can i change my life, families, but also the people around me'

**Generate csv file for Attainment**

In [131]:
attainment_df = theme_datasets["Attainment"]

In [132]:
attainment_sentence_df = pd.DataFrame(columns=['sentence', 'label', 'phrase'])
with open('/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/using_original_method/attainment_sentence_level_dataset.csv','w',newline='', encoding = 'UTF-8') as f:
    writer = csv.writer(f)
    writer.writerow(['sentence','phrases','label'])
    for index,val in attainment_df.iterrows():
        text = val['Essay: Why I am here?'].rstrip('\n\r')
        # using sent_tokenize from nltk to split the paragraphs into sentences.
        temp_text = re.split('\.|\?|\!',text) # text.split('.')
        # annotation
        annotation = val['Attainment']
        if '/%/' in annotation:
            annotation = re.sub('/%/', '. ', annotation)
        # annotation list
        annotation_list = re.split('\.|\?|\!',annotation) #annotation.split('.')
        annotation_strip = [y.rstrip('\r\n') for y in annotation_list if y != '']
        essay_strip = [x for x in temp_text if x !='']
        
        # remove the extra spaces before the sentences.
        annotation_strip = [y.lstrip(" ") for y in annotation_strip]
        essay_strip = [x.lstrip(" ") for x in essay_strip]
        
        # annotate the essay on sentence level.
        for sentence in essay_strip:
            if not (re.search('\w',sentence)):
                continue
            if (len(sentence.split(' ')) < 3):
                continue
            for annotation_phrase in annotation_strip:
                if len(annotation_phrase.split()) > 2:
                    label = get_sentence_label(sentence,annotation_phrase)
                    if label == 1:
                        break
            writer.writerow([sentence.lower(), annotation_strip, label])
            # Create a new DataFrame with the row you want to add
            new_row = pd.DataFrame([{'sentence': sentence.lower(), 'label': label, 'phrase': annotation_strip}])

            # Concatenate the new row to the existing DataFrame
            attainment_sentence_df = pd.concat([attainment_sentence_df, new_row], ignore_index=True)
            # attainment_sentence_df = attainment_sentence_df.append({'sentence' : sentence.lower() , 'label': label , 'phrase':annotation_strip}, ignore_index=True)
            #attainment_sentence_df = attainment_sentence_df.append({'sentence' : sentence , 'label': label , 'phrase':annotation_strip}, ignore_index=True)

In [59]:
attainment_df

,ID CODE,Essay : Why am I here,Attainment - FINAL
0,pilot_study_01,"I am here to get my BS in Physiology, so I can...",I am here to get my BS in physiology so I can ...
8,pilot_study_09,I’m here to learn and to be on the path toward...,I’m here to learn and to be on the path toward...
9,pilot_study_10,I am in SFSU because I want to go to medical s...,I am in SFSU because I want to go to medical s...
11,pilot_study_12,I am here in SFSU because I am the first gener...,My goal is to graduate with a degree B.A./B.S.
13,pilot_study_14,I am here because of my goal in trying to get ...,I am here because of my goal in trying to get ...
...,...,...,...
551,F19.PHYS232.07.23,I'm here because of prerequisite for my comput...,And my dream job is to be a software engineer.
559,F19.PHYS232.09.08,The reason I am here is to do experiments and ...,I am an aspiring Engineer and I love doing ex...
561,F19.PHYS232.09.10,I am in college to further my education. There...,I am studying to become an electrical engineer.
562,F19.PHYS242.05.01,I am here at SFSU for a multitude of reasons. ...,My second is to receive a degree in Engineerin...


In [58]:
attainment_sentence_df

,sentence,label,phrase


In [27]:
pos_labels = len([n for n in attainment_sentence_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(attainment_sentence_df['label']), (pos_labels/len(attainment_sentence_df['label']))*100))


Positive labels present in the dataset : 253  out of 1413 or 17.905166312809627%


In [28]:
train, test = train_test_split(attainment_sentence_df, test_size=0.1, random_state=18, stratify=attainment_sentence_df['label'])

In [29]:
train.to_csv("data/csv/attainment_sentence_level_training_data.csv", encoding='UTF-8', index=False)

In [30]:
test.to_csv("data/csv/attainment_sentence_level_test_data.csv",  encoding='UTF-8', index=False)

In [31]:
print('Class distibution of Training data: {:0.2f}% positive class'.format(len([m for m in train['label'] if m==1])/len(train['label']) *100))
print('positive labels : {} out of total {} datapoints'.format(len([m for m in train['label'] if m==1]), len(train['label'])))

Class distibution of Training data: 17.94% positive class
positive labels : 228 out of total 1271 datapoints


In [32]:
print('Class distibution of Test data: {:0.2f}% positive class'.format(len([m for m in test['label'] if m==1])/len(test['label']) *100))
print('positive labels : {} out of total {} datapoints'.format(len([m for m in test['label'] if m==1]), len(test['label'])))

Class distibution of Test data: 17.61% positive class
positive labels : 25 out of total 142 datapoints


**Generate csv file for First Generation**

In [33]:
first_generation_sentence_df = pd.DataFrame(columns=['sentence', 'label', 'phrase'])
with open('data/csv/first_generation_sentence_level_dataset.csv','w',newline='', encoding = 'UTF-8') as f:
    writer = csv.writer(f)
    writer.writerow(['sentence','phrases','label'])
    for index,val in first_generation_df.iterrows():
        text = val['Essay : Why am I here'].rstrip('\n\r')
        # using sent_tokenize from nltk to split the paragraphs into sentences.
        temp_text = re.split('\.|\?|\!',text) # text.split('.')
        # annotation
        annotation = val['First Gen - FINAL']
        if '/%/' in annotation:
            annotation = re.sub('/%/', '. ', annotation)
        # annotation list
        annotation_list = re.split('\.|\?|\!',annotation) #annotation.split('.')
        annotation_strip = [y.rstrip('\r\n') for y in annotation_list if y != '']
        essay_strip = [x for x in temp_text if x !='']
        
        # remove the extra spaces before the sentences.
        annotation_strip = [y.lstrip(" ") for y in annotation_strip]
        essay_strip = [x.lstrip(" ") for x in essay_strip]
        
        # annotate the essay on sentence level.
        for sentence in essay_strip:
            if not (re.search('\w',sentence)):
                continue
            if (len(sentence.split(' ')) < 3):
                continue
            for annotation_phrase in annotation_strip:
                if len(annotation_phrase.split()) > 2:
                    label = get_sentence_label(sentence,annotation_phrase)
                    if label == 1:
                        break
            writer.writerow([sentence.lower(), annotation_strip, label])
            first_generation_sentence_df = first_generation_sentence_df.append({'sentence' : sentence.lower() , 'label': label , 'phrase':annotation_strip}, ignore_index=True)
            #first_generation_sentence_df = first_generation_sentence_df.append({'sentence' : sentence , 'label': label , 'phrase':annotation_strip}, ignore_index=True)
            

In [34]:
first_generation_sentence_df.shape

(128, 3)

In [35]:
pos_labels = len([n for n in first_generation_sentence_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(first_generation_sentence_df['label']), (pos_labels/len(first_generation_sentence_df['label']))*100))


Positive labels present in the dataset : 19  out of 128 or 14.84375%


In [36]:
train, test = train_test_split(first_generation_sentence_df, test_size=0.2, random_state=18, stratify=first_generation_sentence_df['label'])

In [37]:
train.to_csv("data/csv/first_generation_sentence_level_training_data.csv", encoding='UTF-8', index=False)

In [38]:
test.to_csv("data/csv/first_generation_sentence_level_test_data.csv",  encoding='UTF-8', index=False)

In [39]:
print('Class distibution of Training data: {:0.2f}% positive class'.format(len([m for m in train['label'] if m==1])/len(train['label']) *100))
print('positive labels : {} out of total {} datapoints'.format(len([m for m in train['label'] if m==1]), len(train['label'])))

Class distibution of Training data: 14.71% positive class
positive labels : 15 out of total 102 datapoints


In [40]:
print('Class distibution of Test data: {:0.2f}% positive class'.format(len([m for m in test['label'] if m==1])/len(test['label']) *100))
print('positive labels : {} out of total {} datapoints'.format(len([m for m in test['label'] if m==1]), len(test['label'])))

Class distibution of Test data: 15.38% positive class
positive labels : 4 out of total 26 datapoints
